In [6]:
import pandas as pd

glucose = pd.read_csv("GLUCOSE_training_data_final.csv")

In [7]:
glucose.head()

,experiment_id,story_id,worker_id,submission_time_normalized,worker_quality_rating,selected_sentence_index,story,selected_sentence,1_specificNL,1_specificStructured,...,8_generalStructured,9_specificNL,9_specificStructured,9_generalNL,9_generalStructured,10_specificNL,10_specificStructured,10_generalNL,10_generalStructured,number_filled_in
0,e56c7c3e-4660-40fb-80d0-052d566d676a__4,e56c7c3e-4660-40fb-80d0-052d566d676a,19,20190930,3,4,It was bedtime at our house. Two of the three ...,Finally he becomes tired and falls asleep.,The third kid continues to get out of bed and...,{The third kid}_[subject] {continues}_[verb] {...,...,escaped,escaped,escaped,escaped,escaped,escaped,escaped,escaped,escaped,2
1,b03eba12-db74-4fce-a3cc-c743948ef8c2__4,b03eba12-db74-4fce-a3cc-c743948ef8c2,2,20200115,1,4,Sara was invited to the football game. She sho...,"At the end of the game, her team won.",Sara goes to a football game >Causes/Enabl...,{Sara }_[subject] {goes }_[verb] {to }_[prepos...,...,escaped,escaped,escaped,escaped,escaped,escaped,escaped,escaped,escaped,3
2,7e7aeb03-12d8-4f02-9097-c3273a692f20__3,7e7aeb03-12d8-4f02-9097-c3273a692f20,60,20191230,1,3,Kyle was leaving to pick up his sister from wo...,He decided to check his dirty clothes basket.,Kyle realized his keys were missing >Causes/En...,{Kyle}_[subject] {realized}_[verb] {his keys}_...,...,escaped,escaped,escaped,escaped,escaped,He decided to check his dirty clothes basket f...,{He}_[subject] {decided to check}_[verb] {his ...,Someone_A decided to check Something_A (that h...,{Someone_A}_[subject] {decided to check}_[verb...,5
3,a2bb14bd-9094-4dcc-93a5-bb93b1c308a5__4,a2bb14bd-9094-4dcc-93a5-bb93b1c308a5,142,20191203,2,4,MY DAUGHTER HAD TO DO A SCIENCE PROJECT. SO SH...,SO CHEWING GUM CAN HELP WITH YOUR CONCENTRATIO...,My daughter has a science project >Causes/En...,{My daughter}_[subject] {has }_[verb] {a }_[pr...,...,escaped,Chewing gum can help with concentration >Resul...,{Chewing gum}_[subject] {can help}_[verb] {wit...,Chewing gum can help with concentration >Resul...,{Chewing gum}_[subject] {can help}_[verb] {wit...,escaped,escaped,escaped,escaped,7
4,b8c68631-fa42-40a0-93c7-6739402e2038__4,b8c68631-fa42-40a0-93c7-6739402e2038,19,20191213,3,4,Vivian couldn't stop thinking about that man s...,Vivian spent the rest of her life chasing that...,escaped,escaped,...,escaped,escaped,escaped,escaped,escaped,escaped,escaped,escaped,escaped,2


In [12]:
import re

placeholders = set()
general_cols = [col for col in glucose.columns if "general" in col]

for i, row in glucose.iterrows():
    for col in general_cols:
        rule = row[col]
        placeholders = placeholders.union(set(re.findall("\w+_\w+", rule)))

with open("placeholders.txt", "w") as f:
    f.write("\n".join(placeholders))

In [31]:
import spacy
nlp = spacy.load("en_core_web_sm", exclude=["ner", "parser"])

def has_overlap(specific_text: str, general_text: str, threshold: float) -> bool:
    sp_doc = nlp(specific_text)
    gl_doc = nlp(general_text)

    sp_words = set([token.lemma_.lower() for token in sp_doc])
    gl_words = set([token.lemma_.lower() for token in gl_doc if token.text not in placeholders])

    return (len(sp_words.intersection(gl_words)) / len(gl_words)) > threshold


def has_overlap_with_story(text: str, story: str, threshold: float) -> bool:
    contexts = story.split(".")
    doc = nlp(text)
    words = set([token.lemma_.lower() for token in doc])

    for context in contexts:
        context_doc = nlp(context)
        context_words = set([token.lemma_.lower() for token in context_doc])

        if (len(words.intersection(context_words)) / len(words)) > threshold:
            return True

    return False   

In [32]:
from tqdm import tqdm

SP_GL_THRESHOLD = 0.9
dim_overlaps = [[], [], [], [], [], [], [], [], [], []]

for i, row in tqdm(glucose.iterrows(), total=len(glucose)):
    for dim in range(1, 11):
        specific_rule = row[f"{dim}_specificNL"].strip()
        general_rule = row[f"{dim}_generalNL"].strip()

        if specific_rule == "escaped" or general_rule == "escaped":
            dim_overlaps[dim-1].append([None, None])
        else:
            specific_parts = re.split(">.*>", specific_rule)
            general_parts = re.split(">.*>", general_rule)

            assert len(specific_parts) == len(general_parts) == 2

            overlap = []

            for sp_text, gl_text in zip(specific_parts, general_parts):
                overlap.append(has_overlap(sp_text.strip(), gl_text.strip(), SP_GL_THRESHOLD))

            dim_overlaps[dim-1].append(overlap)

dim_overlap_df = pd.DataFrame()

for dim in range(1, 11):
    dim_overlap_df[f"{dim}_has_overlap_{SP_GL_THRESHOLD}"] = dim_overlaps[dim-1]

dim_overlap_df.to_csv("glucose_rule_overlap.csv")

  1%|          | 432/65522 [00:34<1:26:54, 12.48it/s]


KeyboardInterrupt: 

In [34]:
from tqdm import tqdm

SP_STORY_THRESHOLD = 0.9
dim_story_overlaps = [[], [], [], [], [], [], [], [], [], []]

for i, row in tqdm(glucose.iterrows(), total=len(glucose)):
    for dim in range(1, 11):
        specific_rule = row[f"{dim}_specificNL"].strip()

        if specific_rule == "escaped":
            dim_overlaps[dim-1].append(None)
        else:
            specific_parts = re.split(">.*>", specific_rule)
            overlap = has_overlap(specific_parts[1].strip(), row['story'], SP_STORY_THRESHOLD)
            dim_overlaps[dim-1].append(overlap)

for dim in range(1, 11):
    glucose[f"{dim}_has_story_overlap_{SP_STORY_THRESHOLD}"] = dim_overlaps[dim-1]

  0%|          | 60/65522 [00:04<1:17:31, 14.07it/s]


KeyboardInterrupt: 